In [29]:
import torch.nn as nn

# settings
cfg = {
    'VGG11':[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13':[64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

# model
class VGG(nn.Module):
    def __init__(self, vgg_name, num_classes=10):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
        self.classifier = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for h in cfg:
            if h == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else :
                layers += [nn.Conv2d(in_channels, h, kernel_size=3, padding=1),
                            nn.BatchNorm2d(h),
                            nn.ReLU(inplace=True)]
                in_channels = h
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers) 

# VGGs
def VGG11():
    return VGG('VGG11')

def VGG13():
    return VGG('VGG13')

def VGG16():
    return VGG('VGG16')

def VGG19():
    return VGG('VGG19')

In [46]:
import torch 
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import argparse

# main
def main():
    parser = argparse.ArgumentParser(description='cifar-10 with PyTorch')
    parser.add_argument('--lr', default=0.001, type=float, help='learning rate')
    parser.add_argument('--epoch', default=50, type=int, help='number of epoch tp train for') 
    parser.add_argument('--trainBatchSize', default=128, type=int, help='training batch size')
    parser.add_argument('--testBatchSize', default=128, type=int, help='testing batch size')
    parser.add_argument('--cuda', default=torch.cuda.is_available(), type=bool, help='use cuda or not')
    
    config_list = ['--lr', '0.001', '--epoch', '50', '--trainBatchSize', '128', '--testBatchSize', '128', '--cuda', 'True']
    args = parser.parse_args(config_list) 
    
    solver = Solver(args)
    solver.run()

CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Solver
class Solver(object):
    def __init__(self, config):
        self.model = None
        self.lr = config.lr
        self.epochs = config.epoch
        self.train_batch_size = config.trainBatchSize
        self.test_batch_size = config.testBatchSize
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        self.device = 'cuda' if config.cuda else 'cpu'
        self.train_loader = None
        self.test_loader = None
        
    def print_model(self):
        print(self.model)
        
    def load_data(self):
        train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
        test_transform = transforms.Compose([transforms.ToTensor()])
        train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
        test_set = datasets.CIFAR10(root='./data', train=False, download=False, transform=test_transform)
        self.train_loader = DataLoader(train_set, batch_size=self.train_batch_size, shuffle=True)
        self.test_loader = DataLoader(test_set, batch_size=self.test_batch_size, shuffle=False)
    
    def load_model(self):
        # self.model = LeNet().to(self.device)
        # self.model = AlexNet().to(self.device)
        self.model = VGG11().to(self.device)
        # self.model = VGG13().to(self.device)
        # self.model = VGG16().to(self.device)
        # self.model = VGG19().to(self.device)
        # self.model = GoogLeNet().to(self.device)
        # self.model = resnet18().to(self.device)
        # self.model = resnet34().to(self.device)
        # self.model = resnet50().to(self.device)
        # self.model = resnet101().to(self.device)
        # self.model = resnet152().to(self.device)
        # self.model = DenseNet121().to(self.device)
        # self.model = DenseNet161().to(self.device)
        # self.model = DenseNet169().to(self.device)
        # self.model = DenseNet201().to(self.device)
        # self.model = WideResNet(depth=28, num_classes=10).to(self.device)
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[75, 150], gamma=0.5)
        self.criterion = nn.CrossEntropyLoss().to(self.device)
    
    # train
    def train(self):
        print('Training:')
        self.model.train()
        train_loss = 0.0
        train_correct = 0 
        total = 0 
        
        for ibatch, (images, labels) in enumerate(self.train_loader):
            images, labels = images.to(self.device), labels.to(self.device)
            outputs = self.model(images)
            self.optimizer.zero_grad()
            loss = self.criterion(outputs, labels)
            loss.backward()
            self.optimizer.step()
            train_loss += loss.item()
            _, pred = torch.max(outputs, 1) # second param "1" represents the dimension to be reduced
            total += labels.size(0)
            
            # train_correct incremented by one if predicted right
            # train_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())
            train_correct += (pred == labels).sum().item()
            if ibatch % 99 == 0:
                print('\t{}/{}: loss = {:.4f}, Acc = {:.3f}%'.format(ibatch, len(self.train_loader), train_loss/(ibatch+1), 100. * train_correct/total))
        return train_loss, float(train_correct/total)
    
    # test
    def test(self):
        print('Testing:')
        self.model.eval()
        test_loss = 0.0 
        test_correct = 0 
        total = 0
        
        with torch.no_grad():
            for ibatch, (images, labels) in enumerate(self.test_loader):
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                test_loss += loss.item()
                _, pred = torch.max(outputs, 1)
                total += labels.size(0)
                test_correct += (pred == labels).sum().item()
                if ibatch % 99 == 0:
                    print('\t{}/{}: loss = {:.4f}, Acc = {:.3f}%'.format(ibatch, len(self.test_loader), test_loss/(ibatch+1), 100. * test_correct/total))
        return test_loss, float(test_correct/total)
    
    def save_model(self):
        model_out_path = './model/vgg_cifar10.pth'
        torch.save(self.model, model_out_path)
        print("* Checkpoint saved to {}".format(model_out_path))
        
    # run
    def run(self):
        self.load_data()
        self.load_model() 
        accuracy = 0.
        
        for epoch in range(1, self.epochs + 1):
            self.scheduler.step(epoch)
            print("\n===> epoch: {}/{}".format(epoch, self.epochs))
            train_result = self.train()
            print(train_result)
            test_result = self.test()
            accuracy = max(accuracy, test_result[1])
        print("===> BEST ACC. PERFORMANCE: {:.3f}%".format(accuracy * 100))
        self.save_model()

In [45]:
if __name__ == '__main__':
    main()

Files already downloaded and verified

===> epoch: 1/50
Training:
	0/391: loss = 2.4044, Acc = 7.812%
	99/391: loss = 1.6178, Acc = 38.789%
	198/391: loss = 1.4471, Acc = 45.705%
	297/391: loss = 1.3331, Acc = 50.503%
(489.9504368901253, 0.5383)
Testing:
	0/79: loss = 1.0071, Acc = 68.750%

===> epoch: 2/50
Training:
	0/391: loss = 1.0513, Acc = 57.812%
	99/391: loss = 0.8568, Acc = 69.031%
	198/391: loss = 0.8368, Acc = 69.892%
	297/391: loss = 0.8115, Acc = 71.112%
(310.74872571229935, 0.7187)
Testing:
	0/79: loss = 1.0914, Acc = 64.062%

===> epoch: 3/50
Training:
	0/391: loss = 0.6961, Acc = 78.906%
	99/391: loss = 0.6594, Acc = 76.945%
	198/391: loss = 0.6444, Acc = 77.371%
	297/391: loss = 0.6370, Acc = 77.666%
(245.2729464173317, 0.7803)
Testing:
	0/79: loss = 0.6603, Acc = 75.000%

===> epoch: 4/50
Training:
	0/391: loss = 0.6494, Acc = 76.562%
	99/391: loss = 0.5344, Acc = 81.602%
	198/391: loss = 0.5302, Acc = 81.729%
	297/391: loss = 0.5250, Acc = 81.858%
(205.69178184866905


===> epoch: 33/50
Training:
	0/391: loss = 0.0169, Acc = 100.000%
	99/391: loss = 0.0381, Acc = 98.789%
	198/391: loss = 0.0366, Acc = 98.818%
	297/391: loss = 0.0335, Acc = 98.904%
(12.824751075357199, 0.98934)
Testing:
	0/79: loss = 0.4120, Acc = 89.062%

===> epoch: 34/50
Training:
	0/391: loss = 0.0217, Acc = 99.219%
	99/391: loss = 0.0223, Acc = 99.281%
	198/391: loss = 0.0264, Acc = 99.144%
	297/391: loss = 0.0266, Acc = 99.156%
(11.337791154161096, 0.99054)
Testing:
	0/79: loss = 0.7190, Acc = 86.719%

===> epoch: 35/50
Training:
	0/391: loss = 0.0167, Acc = 100.000%
	99/391: loss = 0.0263, Acc = 99.070%
	198/391: loss = 0.0288, Acc = 98.999%
	297/391: loss = 0.0307, Acc = 98.954%
(11.794409663416445, 0.98978)
Testing:
	0/79: loss = 0.3620, Acc = 92.188%

===> epoch: 36/50
Training:
	0/391: loss = 0.0269, Acc = 100.000%
	99/391: loss = 0.0276, Acc = 99.070%
	198/391: loss = 0.0297, Acc = 99.022%
	297/391: loss = 0.0284, Acc = 99.056%
(11.183280417695642, 0.99044)
Testing:
	0/79

AttributeError: 'Solver' object has no attribute 'save'